<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/translate_clause_attribute_vSB20230209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using spacy to split sentences for translation

In [1]:
!pip install -U spacy

  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.


In [2]:
!python -m spacy download fr_core_news_sm

✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [3]:
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
nlp = spacy.load("fr_core_news_sm")

In [ ]:

# ## loading txt file
# d = '/content/drive/MyDrive/combined_txt_evaluatn.txt'
# d

# ## Reading text from .txt file and storing in a string
# x = ''
# with open(d) as f:
#   for line in f:
#     x = x + line
# print(x)

In [ ]:
!pip install transformers[sentencepiece]

## Importing model
from transformers import pipeline

qa = pipeline("text2text-generation", model='Helsinki-NLP/opus-mt-fr-en')

In [6]:
## Translating attribute #1
x = "Le délai global de paiement est de 50jours"
qa(x)[0]['generated_text']

'The total payment period is 50 days'

In [8]:
## Translating attribute #2
x = "prix révisables."
qa(x)[0]['generated_text']

'revised prices.'

In [9]:
## Translating clause #1
x = "Le délai global de paiement est de 50jours."
qa(x)[0]['generated_text']

'The overall payment period is 50 days.'

In [16]:
## Translating clause #2, which consists of long texts
x = """Dans le délai de paiement sont incluses les phases de réception de la commande et de certification du
règlement par le comptable public.

En cas de dépassement du délai de paiement, des intérêts moratoires sont versés au titulaire, calculés
par application de la formule suivante :

IM = M x J/365 x Taux IM + F ;

Dans laquelle : IM : montant des intérêts moratoires - M : montant TTC de la demande de paiement -
Taux IM : taux de la Banque Centrale Européenne en vigueur majoré de 8 points - J : nombre de jours
calendaires entre la date limite et la date réelle de paiement - F : forfait de 40 € de frais de recouvrement"""


## Tokenizing into sentences and translating
doc = nlp(x)

translated_string = ''

for sentence in doc.sents:
  print(sentence.text)
  print(qa(sentence.text), '\n')
  print("End of sentence...", '\n')

  translated_string = translated_string + ' ' + qa(sentence.text)[0]['generated_text']

Dans le délai de paiement sont incluses
[{'generated_text': 'Within the payment period are included'}] 

End of sentence... 

les phases de réception de la commande et de certification du
règlement par le comptable public.


[{'generated_text': 'the phases of receipt of the order and certification of the settlement by the public accountant.'}] 

End of sentence... 

En cas de dépassement du délai de paiement, des intérêts moratoires sont versés au titulaire, calculés
par application de la formule suivante :

IM = M x J/365 x Taux IM + F ;

Dans laquelle : IM : montant des intérêts moratoires
[{'generated_text': 'In case of exceeding the payment deadline, interest shall be paid to the holder, calculated by applying the following formula: IM = M x J/365 x IM + F rate; In which: IM: amount of interest moratoriums'}] 

End of sentence... 

- M : montant TTC de la demande de paiement -
Taux IM : taux de la Banque Centrale Européenne en vigueur majoré de 8 points
[{'generated_text': '- M : T

In [17]:
translated_string

' Within the payment period are included the phases of receipt of the order and certification of the settlement by the public accountant. In case of exceeding the payment deadline, interest shall be paid to the holder, calculated by applying the following formula: IM = M x J/365 x IM + F rate; In which: IM: amount of interest moratoriums - M : TTC amount of payment request - IM rate : rate of the European Central Bank in force plus 8 points - J: number of calendar days between the deadline and the actual payment date - F: € 40 recovery fee package'

In [ ]:
## Tokenization
doc = nlp(x)
print(type(doc.sents))
for sentence in doc.sents:
  print(sentence.text)
  print(qa(sentence.text))

In [10]:
translated_df = pd.DataFrame()

for i, token in enumerate(sentence_list):
  print(i)
  try:
    trans = qa(sentence_list[i])
    # translated_text = translated_text + '\n' + trans[0]['generated_text']
    translated_df = pd.concat([translated_df, pd.DataFrame({'raw_text':[sentence_list[i]], 'translated_text' :[trans[0]['generated_text']]})])
  except Exception as e:
    print(e)
    print(f'failed for {i}, length:{len(sentence_list[i])}')
  


NameError: ignored

## End of spacy tokenization testing

In [6]:
x = """5 - Durée @t CÉlaiS d'EXÉCUÎÏON 467s5recrmensenenenennennnen rrr franmenmennmee srr 15
5.1 - Durée globale prévisionnelle des prestationSs ………………….…………..…..…………rirseriecneacnrancensarensrennereues 15
5.2 - Délai d'EXÉCUtION ….....…...…….……………rercesneerrecesmenarcenrenrarcenmententencenscentencennencensensensendessveseenrs 15
5.3 - RECONQUCtION …....…..……..…..….......rcrcenrenrencenrenrenererancencenreneesercncereneneneare nn en scc se e es 15
LI à RE E E E 16
6.1 < Application d'une EPPR L rsreccecrmensenenenenen en rrrs rr rrtithanmnns cnseranseseeneneee HRN 16
6.2 - Caractéristiques des prix pratiquéS.….….…….…….…..…….…..…eemnennennnnnünnnnnnnnnnnnnnnnnnmn 16
6.3 - Modalités de variation des priX ……………..….………………crrrrenenmnenntnnnmnneneensenssranse cerrrsresen es 16
7— RECOUrS QU'CALAIOQUE … cruevemenenennçänenenmnnnenenen nn nn eqn nn aontntsn dn 17
8 - Garanties FinancCières ……….…….……….…………crrrrrecreriereceenrerernres e rcenrrerarenvenre ce en aneaeenas se enee nn 17
9 - Modalités de règlement des COMPptES.……..….…....…..….…………rrrcrrsererrenarcrrrrrrrrrarenrasrran ccn narerane en 17
9.1 - Acomptes et paiements partiels définitifs..…….….………..…..……rienennsmnnmnnnnnnnnnnnnnnnn*nnn*n* 17
9.2 - Présentation des demandes de paiement …………………….….……..….….……rrrrrerernnsnenennntnün+nnnnnnnn*nnnnn 17
9,3 - Délai-global d& PaleMENt.. v1remenmenmeremenenenenen en seesercnsesiranmanannenmnnnnnnnnnnnnnn“nqgüttntetñnnnçnt 18
9.4 - Paiement des cotraitants……….….………..………crererrecereereaneenensenereererareerrre es en cen rrs rrn 18
10 - Conditions d'exécution des prestations …………….………….…….….….renennnnnnnnnnnnnnnnnnnnn 18
11 - Droit de propriété industrielle et intellectuelle.………….…….…..….…….….…….….…rerrreserrenrenrarererranranenerenss 18
12 - Constatation de l'exécution des prestations …………………….…….….….…errsennnnmen£tnnnnnnnnnnnnnnnn 19
12.1 - VÉrificatioNnS…....…..………….…...…rererrrercenrerrcrmernrrecernerenn en srenren sn ne se rc e nn en e 19
13 - Garantie des prestations …………….……..…….…………crerereermenenennen ansenaeeresreraenarenere en en 19
14 - PÉnalitéS ……..….....…....……rcssrrerscercecsercerrererearenenenrereerrne en e rrrn en re en 19
14;1 = PÉnalités Q@ F@lArOr.2>cemememmenanÊnnenmnenensnnnnmÊen nccnen crcs ianennneanmenenenenn s."""

In [7]:
doc = nlp(x)

In [ ]:
doc = nlp(x)

for sentence in doc.sents:
  print(len(sentence))

02/08: Using sentence tokenizer and translating French to English

In [ ]:
!pip install transformers[sentencepiece]

In [2]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
## loading txt file
d = '/content/drive/MyDrive/combined_txt_evaluatn.txt'
d

'/content/drive/MyDrive/combined_txt_evaluatn.txt'

In [ ]:
## Reading text from .txt file and storing in a string
x = ''
with open(d) as f:
  for line in f:
    x = x + line
print(x)

In [5]:
 import nltk
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
## Tokenizing string into sentences
from nltk.tokenize import sent_tokenize
sentence_list = sent_tokenize(x, language = "french")

In [7]:
## Importing model
from transformers import pipeline

qa = pipeline("text2text-generation", model='Helsinki-NLP/opus-mt-fr-en')


In [11]:
qa(sentence_list[2])[0]

[{'generated_text': '18.18.00 : GENERAL CONSUMPTION MEDICAL DEPOSITS D-ABORD PARENm.'}]

In [8]:
sentence_list_new = []
for i in sentence_list:
  sentence_list_new.extend(i.split('\n\n'))

In [9]:
print(len(sentence_list_new), len(sentence_list))

480 131


In [10]:
import pandas as pd

In [ ]:
old_sentence = sentence_list
sentence_list = sentence_list_new

# translated_text = ''
print("Loop will run for", len(sentence_list))

translated_df = pd.DataFrame()

for i, token in enumerate(sentence_list):
  print(i)
  try:
    trans = qa(sentence_list[i])
    # translated_text = translated_text + '\n' + trans[0]['generated_text']
    translated_df = pd.concat([translated_df, pd.DataFrame({'raw_text':[sentence_list[i]], 'translated_text' :[trans[0]['generated_text']]})])
  except Exception as e:
    print(e)
    print(f'failed for {i}, length:{len(sentence_list[i])}')
  


In [15]:
## Saving pandas dataframe
translated_df.to_csv('/content/drive/MyDrive/combined_txt_evaluatn_translated.csv', index=False)

In [13]:
sentence_list[12]

"5 - Durée @t CÉlaiS d'EXÉCUÎÏON 467s5recrmensenenenennennnen rrr franmenmennmee srr 15\n5.1 - Durée globale prévisionnelle des prestationSs ………………….…………..…..…………rirseriecneacnrancensarensrennereues 15\n5.2 - Délai d'EXÉCUtION ….....…...…….……………rercesneerrecesmenarcenrenrarcenmententencenscentencennencensensensendessveseenrs 15\n5.3 - RECONQUCtION …....…..……..…..….......rcrcenrenrencenrenrenererancencenreneesercncereneneneare nn en scc se e es 15\nLI à RE E E E 16\n6.1 < Application d'une EPPR L rsreccecrmensenenenenen en rrrs rr rrtithanmnns cnseranseseeneneee HRN 16\n6.2 - Caractéristiques des prix pratiquéS.….….…….…….…..…….…..…eemnennennnnnünnnnnnnnnnnnnnnnnnmn 16\n6.3 - Modalités de variation des priX ……………..….………………crrrrenenmnenntnnnmnneneensenssranse cerrrsresen es 16\n7— RECOUrS QU'CALAIOQUE … cruevemenenennçänenenmnnnenenen nn nn eqn nn aontntsn dn 17\n8 - Garanties FinancCières ……….…….……….…………crrrrrecreriereceenrerernres e rcenrrerarenvenre ce en aneaeenas se enee nn 17\n9 - M

In [ ]:
!pip install googletrans==3.1.0a0

In [21]:
from googletrans import Translator
translator = Translator()
# result = translator.translate('Mikä on nimesi', src = 'fr', dest='en')

translated_df['google_trans_output'] = translated_df['raw_text'].apply(lambda x: translator.translate(x, src = 'fr', dest='en'))




In [22]:
translated_df.to_csv('/content/drive/MyDrive/combined_txt_evaluatn_translated_w_gtrans.csv', index=False)

In [17]:
qa(sentence_list[11])

[{'generated_text': 'Benefits are subject to security measures in accordance with section 5.3 of the CCAG-FCS.'}]

In [19]:
len(sentence_list[10])

12952

In [20]:
len(sentence_list[9])

7

In [16]:
qa("- L'acte d'engagement (AE) et ses éventuelles annexes ;")

[{'generated_text': '- the Act of Commitment (AE) and any annexes thereto;'}]

In [ ]:
# translated_text = ''

# # list_temp = sentence_list[1]

# for i in sentence_list[1:3]:
#   trans_i = qa(i)
  
#   translated_text = translated_text + '/n' + trans_i[0]['generated_text']
  


## Testing #2

In [ ]:
pip install dl-translate

In [ ]:
import dl_translate as dlt

mt = dlt.TranslationModel()  # Slow when you load it for the first time

text_hi = "Les prestations sont réparties en 119 lots."
mt.translate(text_hi, source=dlt.lang.FRENCH, target=dlt.lang.ENGLISH)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'The benefits are divided into 119 lots.'

## Installing libraries

In [ ]:
!pip install torch trochvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement trochvision (from versions: none)
ERROR: No matching distribution found for trochvision


In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

## Importing libraries

In [ ]:
## To link Google Drive with collab - one time thing
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
import pandas as pd


In [ ]:
def prepare_translation_datasets(data_path):
    with open(os.path.join(data_path, "train.trg"), "r", encoding="utf-8") as f:
        sinhala_text = f.readlines()
        sinhala_text = [text.strip("\n") for text in sinhala_text]

    with open(os.path.join(data_path, "train.src"), "r") as f:
        english_text = f.readlines()
        english_text = [text.strip("\n") for text in english_text]

    data = []
    for sinhala, english in zip(sinhala_text, english_text):
        data.append(["translate sinhala to english", sinhala, english])
        data.append(["translate english to sinhala", english, sinhala])

    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    with open(os.path.join(data_path, "test.trg"), "r", encoding="utf-8") as f:
        sinhala_text = f.readlines()
        sinhala_text = [text.strip("\n") for text in sinhala_text]

    with open(os.path.join(data_path, "test.src"), "r") as f:
        english_text = f.readlines()
        english_text = [text.strip("\n") for text in english_text]

    data = []
    for sinhala, english in zip(sinhala_text, english_text):
        data.append(["translate sinhala to english", sinhala, english])
        data.append(["translate english to sinhala", english, sinhala])

    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    return train_df, eval_df

In [ ]:
train_df, eval_df = prepare_translation_datasets("/content/drive/MyDrive/LanguageTranslation/eng_sin")

In [ ]:
# import os
# os.getcwd()


'/content'

In [ ]:
# os.system('ls')

0

In [ ]:
train_df

,prefix,input_text,target_text
0,translate sinhala to english,මෙය සිදු වන්නේ කවදාද?,When will this happen?
1,translate english to sinhala,When will this happen?,මෙය සිදු වන්නේ කවදාද?
2,translate sinhala to english,දුවන්න !,Run!
3,translate english to sinhala,Run!,දුවන්න !
4,translate sinhala to english,- අනිද්දට,- The day after tomorrow.
...,...,...,...
2255049,translate english to sinhala,"As those words indicate, some Christians will ...",ඒ විරුද්ධවාදිකම් ආගමික නැත්නම් දේශපාලන නයකයන්ග...
2255050,translate sinhala to english,එතකොට ඔයා මොකද කරන්නේ?,Then what are you to do?
2255051,translate english to sinhala,Then what are you to do?,එතකොට ඔයා මොකද කරන්නේ?
2255052,translate sinhala to english,"ඔහු ජාන විද්‍යාඥයෙක්, මානව ජානවිද්‍යාව ගැන විශ...","Only that he's a genecist, an expert on human ..."


In [ ]:
eval_df

,prefix,input_text,target_text
0,translate sinhala to english,මම ගෙදර යනවා,I'm going home.
1,translate english to sinhala,I'm going home.,මම ගෙදර යනවා
2,translate sinhala to english,මා තරුණයි.,I am young.
3,translate english to sinhala,I am young.,මා තරුණයි.
4,translate sinhala to english,හැම දෙයක්ම හරිම අමාරුයි,Everything was just so hard.
...,...,...,...
85,translate english to sinhala,I am young.,මං තරුණයි.
86,translate sinhala to english,මං ඔයාට ආදරෙයි.,I love you.
87,translate english to sinhala,I love you.,මං ඔයාට ආදරෙයි.
88,translate sinhala to english,උඹ නැඟිටිනකොට මං මෙහේ නෑ.,"As you wake up, I won't be here anymore."


In [ ]:
baseDir = '/content/drive/MyDrive/LanguageTranslation/eng_sin/'

In [ ]:
train_df.to_csv(f"{baseDir}train.tsv", sep="\t")
eval_df.to_csv(f"{baseDir}eval.tsv", sep="\t")

In [ ]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_df = pd.read_csv(f"{baseDir}/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv(f"{baseDir}/eval.tsv", sep="\t").astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 20
model_args.eval_batch_size = 20
model_args.num_train_epochs = 1
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Sinhala-English Translation"

model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=False)

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

In [ ]:
# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/2255054 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprec

KeyboardInterrupt: ignored

## Training the model could take hours, hence we used model fine-tuned on the Sinhalese-English dataset and checked its working

In [ ]:
from simpletransformers.t5 import T5Model

model = T5Model("mt5", "thilina/mt5-sinhalese-english", use_cuda=False)
print(model.predict(["Let's translate!"]))

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['අපි පරිවර්තනය කරමු!']


In [ ]:
print(model.predict(["Model trained."]))

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['ආකෘති පුහුණු කරලා.']


## End of Sample model building exercise

## Using pre-trained model to get English-Italian translations
link: https://stackoverflow.com/questions/60513592/how-to-use-huggingface-t5-model-to-test-translation-task

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')

model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)

input = "How are you feeling"

# You can also use "translate English to French" and "translate English to Romanian"
input_ids = tokenizer("translate English to Italian: "+input, return_tensors="pt").input_ids  # Batch size 1

outputs = model.generate(input_ids)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded)

Wie fühlt man sich


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
import os
os.getcwd()

'/content'

## Testing language translation

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

## Downloading data for Italian - English translation

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=3806b5a99ebd634a3cede66d0b7cb7083bb91cab011cdb0d08efeaa9275edca2
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [ ]:
import wget
wget.download('https://object.pouta.csc.fi/Tatoeba-Challenge-v2021-08-07/eng-ita.tar')

'eng-ita.tar'

### PRe-trained translation model vF

In [ ]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

qa = pipeline("text2text-generation", model='Helsinki-NLP/opus-mt-it-en')


/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
!pip install sacremoses


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e4e16b660f1b1c57cbced4471854954d10dad1ce2c9eb9cd8e191f8b111eb3a1
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
qa("MODELLO DI CONTRATTO DI VENDITA INTERNAZIONALE(CI GLOBALNEGOTIATOR Business Publications Scarichi l'esempio e modello di Contratto di Vendita Internazionale in formato Word. Compili gli spazi in bianco e scelga le clausole di questo contratto che più si adattano alle sue necessità. Il Contratto serve per la vendita di vari prodotti (materie prime, componenti industriali, beni di consumo, macchinari, ecc.). Il Contratto è disponibile anche in inglese.")

[{'generated_text': 'MODEL OF INTERNATIONAL SALE AGREEMENT(CI GLOBALNEGOTIATOR Business Publications Download the example and model of International Sale Agreement in Word format. Fill in the blank spaces and choose the clauses of this contract that best suit your needs. The Contract serves for the sale of various products (raw materials, industrial components, consumer goods, machinery, etc.).The Contract is also available in English.'}]

In [ ]:
qa("Scarichi l’esempio e modello di Contratto di Vendita Internazionale in formato Word. Compili gli spazi in bianco e scelga le clausole di questo contratto che più si adattano alle sue necessità. Il Contratto serve per la vendita di vari prodotti (materie prime, componenti industriali, beni di consumo, macchinari, ecc.). Il Contratto è disponibile anche in inglese.")

[{'generated_text': 'Download the example and model of International Sales Agreement in Word format. Fill out the blank spaces and choose the clauses of this contract that best suit your needs. The Contract serves for the sale of various products (raw materials, industrial components, consumer goods, machinery, etc.). The Contract is also available in English.'}]

In [ ]:
qa("……………………. [denominazione sociale della ditta], con sede in ………………… [indirizzo, città e Paese] e numero di registrazione fiscale......................... in persona del legale rappresentante Sig./Sig.ra .................................................... [nome, cognome, carico] (in seguito denominato il Venditore")

[{'generated_text': '[name of company], based on ...................... [address, city and country] and tax registration number................................................. in the person of the legal representative Mr./Mr. .................................................... [name, surname, cargo] (hereinafter referred to as the Seller'}]

In [ ]:
qa("Con il presente Contratto, il Venditore si impegna a fornire e il Compratore si impegna ad acquistare:")

[{'generated_text': 'With this Agreement, the Seller undertakes to provide and the Buyer undertakes to purchase:'}]

In [ ]:
## loading txt file
d = '/content/drive/MyDrive/Combined_text.txt'

In [ ]:
# x = ''
# with open(d) as f:
#   [print(line.strip(), 'STOP') for line in f.readlines()]

#     # trans = qa(line.strip()) for line in f.readlines())
#     # [print(trans)]

In [ ]:
## Reading text from .txt file and storing in a string
x = ''
with open(d) as f:
  for line in f:
    # print(line.strip())
    # print('STOP')
    x = x + line


In [ ]:
x

'\nMODELLO DI CONTRATTO DI VENDITA INTERNAZIONALE\n\n(CI GLOBALNEGOTIATOR\nBusiness Publications\n\nScarichi l\'esempio e modello di Contratto di Vendita Internazionale in formato Word.\nCompili gli spazi in bianco e scelga le clausole di questo contratto che più si adattano\nalle sue necessità. Il Contratto serve per la vendita di vari prodotti (materie prime,\ncomponenti industriali, beni di consumo, macchinari, ecc.). Il Contratto è disponibile\nanche in inglese.\n\nCONTRATTO DI VENDITA INTERNAZIONALE\n\nDATA: ...iiiiiii\n\nTRA\n\nceri [denominazione sociale della ditta], con sede in ..................... [indirizzo,\ncittà e Paese] e numero di registrazione fiscale............ in persona del legale\nrappresentante Sig./Sig.ra i... [nome, cognome, carico] (in seguito\n\ndenominato "il Venditore"),\n\nE\n\ncc [denominazione sociale della ditta], con sede in ............................... [indirizzo,\ncittà e Paese] e numero di registrazione fiscale ............................... , 

In [ ]:
## Splitting line/paragraph, translating one by one and saving to a master string
translated_master = ''

for i in range(x.count('\n\n')+1):
  print(i)
  # print(x.split('\n\n')[i])
  
  ## Splitting string by double line terminator
  y = x.split('\n\n')[i]
  
  ## Translating string
  trans = qa(y)
  # print("Translated is:", qa(y)[0]['generated_text'])
  
  translated_master = translated_master + '\n'+ trans[0]['generated_text']


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [ ]:
## Saving translated text to a .txt file
with open('/content/drive/MyDrive/translated.txt', 'w') as f:
  f.write(translated_master)
  f.close()


In [ ]:
# translated_master = ''

# for i in range(x.count('\n\n')):
#   print(i)
#   # print(x.split('\n\n')[i])
  
#   ## Splitting string by double line terminator
#   y = x.split('\n\n')[i]
  
#   ## Translating string
#   trans = qa(y)
#   # print("Translated is:", qa(y)[0]['generated_text'])
  
#   translated_master = translated_master + '\n'+ trans[0]['generated_text']
#   if i == 5:
#     break

In [ ]:
translated_master

'\nMODEL OF INTERNATIONAL SALE CONTRACT\n(CI GLOBALNEGOTIATOR Business Publications)\nDownload the example and model of International Sales Agreement in Word format. Fill in the blank spaces and choose the clauses of this contract that best suit your needs. The Contract serves for the sale of various products (raw materials, industrial components, consumer goods, machinery, etc.). The Contract is also available in English.\nINTERNATIONAL SALE AGREEMENT\nDATE: ...iiiiiii\n_\n[name of company], based on ..................... [address, city and country] and tax registration number.............. in the person of the legal representative Mr./Mr. i... [name, surname, cargo] (hereinafter:\nreferred to as "the Seller"),\nAnd\n[name of company], based on ................................................... [address, city and country] and tax registration number ................................................... , in the person of the legal representative Mr./Mr. iii [name, surname, cargo] (in\n

## Translating text using google translate

In [ ]:
## Installation
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17414 sha256=3d419c289a452893fc06d40fbdbe95b09627c9c5fb6af5c0ae4db7b352c3d879
  Stored in directory: /root/.cache/pip/wheels/a9/25/af/715361fa79594524c89c75d293def652045f3fdce6ca398712
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [ ]:
import googletrans

## Checking supported languages
print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [ ]:
## Importing Translator class
from googletrans import Translator

In [ ]:
## Creating an object of translator class
translator = Translator()

In [ ]:
 text1 = "MODELLO DI CONTRATTO DI VENDITA INTERNAZIONALE(CI GLOBALNEGOTIATOR Business Publications Scarichi l'esempio e modello di Contratto di Vendita Internazionale in formato Word. Compili gli spazi in bianco e scelga le clausole di questo contratto che più si adattano alle sue necessità. Il Contratto serve per la vendita di vari prodotti (materie prime, componenti industriali, beni di consumo, macchinari, ecc.). Il Contratto è disponibile anche in inglese."

In [ ]:
## Testing google translator on Italian text
result = translator.translate(text1)

print("Source language:", result.src)
print("Source destination:",result.dest)
print("Source origin:",result.origin)
print("Source text:",result.text)
print("Source pronunciation:",result.pronunciation)

Source language: it
Source destination: en
Source origin: MODELLO DI CONTRATTO DI VENDITA INTERNAZIONALE(CI GLOBALNEGOTIATOR Business Publications Scarichi l'esempio e modello di Contratto di Vendita Internazionale in formato Word. Compili gli spazi in bianco e scelga le clausole di questo contratto che più si adattano alle sue necessità. Il Contratto serve per la vendita di vari prodotti (materie prime, componenti industriali, beni di consumo, macchinari, ecc.). Il Contratto è disponibile anche in inglese.
Source text: International sales contract model (Ci Globalnegotitorler Business Publications downloads the example and model of international sales contract in Word format. Fill out the spaces in white and choose the clauses of this contract that most adapt to its needs. The contract serves forThe sale of various products (raw materials, industrial components, consumer goods, machinery, etc.). The contract is also available in English.
Source pronunciation: None


In [ ]:
## Italian txt file path
d = '/content/drive/MyDrive/Combined_text.txt'

## Reading text from .txt file and storing in a string
x = ''
with open(d) as f:
  for line in f:
    # print(line.strip())
    # print('STOP')
    x = x + line


## Splitting line/paragraph, translating one by one and saving to a master string
translated_master = ''

for i in range(x.count('\n\n')+1):
  print(i)
  # print(x.split('\n\n')[i])
  
  ## Splitting string by double line terminator
  y = x.split('\n\n')[i]
  
  ## Translating string
  result = translator.translate(y)
  # print("Translated is:", qa(y)[0]['generated_text'])
  
  translated_master = translated_master + '\n'+ result.text


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [ ]:
x

'\nMODELLO DI CONTRATTO DI VENDITA INTERNAZIONALE\n\n(CI GLOBALNEGOTIATOR\nBusiness Publications\n\nScarichi l\'esempio e modello di Contratto di Vendita Internazionale in formato Word.\nCompili gli spazi in bianco e scelga le clausole di questo contratto che più si adattano\nalle sue necessità. Il Contratto serve per la vendita di vari prodotti (materie prime,\ncomponenti industriali, beni di consumo, macchinari, ecc.). Il Contratto è disponibile\nanche in inglese.\n\nCONTRATTO DI VENDITA INTERNAZIONALE\n\nDATA: ...iiiiiii\n\nTRA\n\nceri [denominazione sociale della ditta], con sede in ..................... [indirizzo,\ncittà e Paese] e numero di registrazione fiscale............ in persona del legale\nrappresentante Sig./Sig.ra i... [nome, cognome, carico] (in seguito\n\ndenominato "il Venditore"),\n\nE\n\ncc [denominazione sociale della ditta], con sede in ............................... [indirizzo,\ncittà e Paese] e numero di registrazione fiscale ............................... , 

In [ ]:
translated_master

'\nInternational sales contract model\n(CI GLOBALNEGOTIATOR\nBusiness Publications\nDownload the example and model of international sales contract in Word format.\nFill out the spaces in white and choose the clauses of this contract that most adapt\nto his needs.The contract is used for the sale of various products (raw materials,\nindustrial components, consumer goods, machinery, etc.).The contract is available\nalso in English.\nInternational sales contract\nDATA: ...iiiiiii\nBETWEEN\nCeri [social denomination of the company], based in ..................... [address,\nCity and country] and tax registration number ............ in the person of the lawyer\nrepresentative Mrs./sig.ra i ... [name, surname, load] (later\ncalled "the seller"),\nE\ncc [social denomination of the company], based in ............................... [address,\nCity and country] and tax registration number ..............................., in the person of the lawyer\nrepresentative Mrs./sig.ra III [name, surname

## Testing another language translation model (model not working good) 

In [ ]:
## Testing another translation-model
from transformers import pipeline

qa = pipeline("text2text-generation", model='SEBIS/legal_t5_small_trans_it_en')
qa("In seguito all' evento di estinzione del")




Downloading:   0%|          | 0.00/823 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/840k [00:00<?, ?B/s]

[{'generated_text': 'Following the extinction of the holding of the holding of the holding of the holding of'}]

In [ ]:
qa("……………………. [denominazione sociale della ditta], con sede in ………………… [indirizzo, città e Paese] e numero di registrazione fiscale......................... in persona del legale rappresentante Sig./Sig.ra .................................................... [nome, cognome, carico] (in seguito denominato il Venditore")


[{'generated_text': '(name, surname, lo load, lo lo lo lo lo lo (hereinafter referred'}]

## Testing hugging face models (Ignore)

In [ ]:
from transformers import pipeline

qa = pipeline("text2text-generation", model='it5/it5-base-translate-italian-to-english')
qa("In seguito all' evento di estinzione del")


OSError: ignored

In [ ]:
from transformers import FlaxT5ForConditionalGeneration, TFT5ForConditionalGeneration

model_flax = FlaxT5ForConditionalGeneration.from_pretrained("gsarti/it5-small")
model_tf = TFT5ForConditionalGeneration.from_pretrained("gsarti/it5-small")

ImportError: ignored

In [ ]:
from transformers import AutoTokenzier, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("gsarti/it5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("gsarti/it5-small")


ImportError: ignored

In [ ]:
from transformers import AutoConfig    
from transformers import T5Tokenizer, T5Model    
model_name = "t5-small"    
config = AutoConfig.from_pretrained(model_name)    
tokenizer = T5Tokenizer.from_pretrained(model_name)    
model = T5Model.from_pretrained(model_name)    
model_raw = T5Model(config)

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

qa = pipeline("text2text-generation", model='it5/it5-base-question-answering')
qa("convertire l'italiano in inglese Che giorno è oggi")


[{'generated_text': 'che giorno è oggi'}]